# **ALL MODELS - POSTa GAN**

## **Import Dependencies**

In [1]:
# !rm -r /kaggle/working

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Number of devices: 2


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers as L
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [4]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# **Data Loader**

In [5]:
# DataLoader
train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adverserial-lisa/gr2',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='training',
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adverserial-lisa/gr2',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='validation',
)

Found 198857 files belonging to 35 classes.
Using 159086 files for training.
Found 198857 files belonging to 35 classes.
Using 39771 files for validation.


In [6]:
train_data.as_numpy_iterator().__next__()[0].shape

(128, 81, 81, 3)

In [7]:
train_data.as_numpy_iterator().__next__()[1].shape

(128, 35)

# **Model Prep**

In [8]:
# Create custom F1 Score Metrics

from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [9]:
steps_per_epoch = len(train_data)
EPOCHS = 128

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.000001)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=8,# if val loss decreases for 8 epochs in a row, stop training
                                                  restore_best_weights = True)

# **Model Creations**

In [10]:
# Model

def create_model_1(shape=(81, 81, 3)):
    base_model = EfficientNetV2B0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetV2B0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [11]:
# Model

def create_model_2(shape=(81, 81, 3)):
    base_model = EfficientNetB0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetB0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [12]:
# Model

def create_model_3(shape=(81, 81, 3)):
    base_model = InceptionV3(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="InceptionV3")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [13]:
# Model

def create_model_4(shape=(81, 81, 3)):
    base_model = ResNet50(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="ResNet50")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [14]:
# Model

def create_model_5(shape=(81, 81, 3)):
    base_model = Xception(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
    for layer in base_model.layers[-3:]:
#         print(layer)
        layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(512, activation="relu", name="Dense-1")(x)
    x = L.Dropout(0.4, name="Final-Dropout")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="Xception")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [15]:
# Model

def create_model_6(shape=(81, 81, 3)):
    base_model = MobileNetV2(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(35, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="MobileNetV2")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.SGD(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

# **Train Models**

In [16]:
model_1 = create_model_1()

# Fit the model
history_1 = model_1.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_1.save("/kaggle/working/EfficientNetV2B0.h5")
pd.DataFrame(history_1.history).to_csv("/kaggle/working/History-EfficientNetV2B0.csv")

24274472/24274472 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 06:28:48.303502: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetV2B0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1243/1243 [==============================] - 240s 185ms/step - loss: 0.3597 - accuracy: 0.9286 - auc: 0.9982 - recall: 0.8453 - precision: 0.9895 - f1_score: 0.8974 - val_loss: 0.1201 - val_accuracy: 0.9790 - val_auc: 0.9996 - val_recall: 0.9594 - val_precision: 0.9931 - val_f1_score: 0.9759 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 59s 47ms/step - loss: 0.1154 - accuracy: 0.9789 - auc: 0.9997 - recall: 0.9600 - precision: 0.9928 - f1_score: 0.9760 - val_loss: 0.0792 - val_accuracy: 0.9857 - val_auc: 0.9998 - val_recall: 0.9746 - val_precision: 0.9948 - val_f1_score: 0.9845 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 59s 47ms/step - loss: 0.0829 - accuracy: 0.9848 - auc: 0.9998 - recall: 0.9724 - precision: 0.9942 - f1_score: 0.9832 - val_loss: 0.0630 - val_accuracy: 0.9880 - val_auc: 0.9998 - val_recall: 0.9801 - val_precision: 0.9953 - val_f1_score: 0.9876 - lr: 0.0100
Epoch 4/128
1243/1243 [==============================] - 60s 4

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
model_2 = create_model_2()

# Fit the model
history_2 = model_2.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_2.save("/kaggle/working/EfficientNetB0.h5")
pd.DataFrame(history_2.history).to_csv("/kaggle/working/History-EfficientNetB0.csv")

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 08:36:30.902754: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetB0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1243/1243 [==============================] - 76s 56ms/step - loss: 0.5324 - accuracy: 0.8870 - auc: 0.9961 - recall: 0.7572 - precision: 0.9826 - f1_score: 0.8373 - val_loss: 0.2305 - val_accuracy: 0.9536 - val_auc: 0.9991 - val_recall: 0.9046 - val_precision: 0.9858 - val_f1_score: 0.9433 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 66s 53ms/step - loss: 0.2153 - accuracy: 0.9554 - auc: 0.9992 - recall: 0.9124 - precision: 0.9860 - f1_score: 0.9476 - val_loss: 0.1585 - val_accuracy: 0.9673 - val_auc: 0.9994 - val_recall: 0.9386 - val_precision: 0.9882 - val_f1_score: 0.9627 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 66s 53ms/step - loss: 0.1619 - accuracy: 0.9669 - auc: 0.9995 - recall: 0.9370 - precision: 0.9883 - f1_score: 0.9619 - val_loss: 0.1287 - val_accuracy: 0.9740 - val_auc: 0.9996 - val_recall: 0.9511 - val_precision: 0.9894 - val_f1_score: 0.9698 - lr: 0.0100
Epoch 4/128
1243/1243 [==============================] - 67s 53m

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1243/1243 [==============================] - 67s 53ms/step - loss: 0.0430 - accuracy: 0.9920 - auc: 0.9999 - recall: 0.9865 - precision: 0.9959 - f1_score: 0.9912 - val_loss: 0.0472 - val_accuracy: 0.9889 - val_auc: 0.9998 - val_recall: 0.9841 - val_precision: 0.9938 - val_f1_score: 0.9889 - lr: 0.0100
Epoch 27/128
1243/1243 [==============================] - 66s 53ms/step - loss: 0.0421 - accuracy: 0.9920 - auc: 0.9999 - recall: 0.9869 - precision: 0.9960 - f1_score: 0.9914 - val_loss: 0.0465 - val_accuracy: 0.9889 - val_auc: 0.9998 - val_recall: 0.9844 - val_precision: 0.9937 - val_f1_score: 0.9890 - lr: 0.0100
Epoch 28/128
1243/1243 [==============================] - 66s 53ms/step - loss: 0.0407 - accuracy: 0.9922 - auc: 0.9999 - recall: 0.9873 - precision: 0.9964 - f1_score: 0.9918 - val_loss: 0.0457 - val_accuracy: 0.9889 - val_auc: 0.9998 - val_recall: 0.9847 - val_precision: 0.9936 - val_f1_score: 0.9891 - lr: 0.0100
Epoch 29/128
1243/1243 [==============================] - 66s 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1243/1243 [==============================] - 68s 54ms/step - loss: 0.0231 - accuracy: 0.9962 - auc: 1.0000 - recall: 0.9939 - precision: 0.9979 - f1_score: 0.9959 - val_loss: 0.0339 - val_accuracy: 0.9916 - val_auc: 0.9998 - val_recall: 0.9885 - val_precision: 0.9946 - val_f1_score: 0.9915 - lr: 0.0100
Epoch 66/128
1243/1243 [==============================] - 68s 54ms/step - loss: 0.0230 - accuracy: 0.9963 - auc: 1.0000 - recall: 0.9937 - precision: 0.9980 - f1_score: 0.9959 - val_loss: 0.0337 - val_accuracy: 0.9915 - val_auc: 0.9998 - val_recall: 0.9886 - val_precision: 0.9947 - val_f1_score: 0.9916 - lr: 0.0100
Epoch 67/128
1243/1243 [==============================] - 66s 53ms/step - loss: 0.0226 - accuracy: 0.9963 - auc: 1.0000 - recall: 0.9939 - precision: 0.9980 - f1_score: 0.9959 - val_loss: 0.0336 - val_accuracy: 0.9916 - val_auc: 0.9998 - val_recall: 0.9887 - val_precision: 0.9945 - val_f1_score: 0.9916 - lr: 0.0100
Epoch 68/128
1243/1243 [==============================] - 66s 

In [18]:
model_3 = create_model_3()

# Fit the model
history_3 = model_3.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_3.save("/kaggle/working/InceptionV3.h5")
pd.DataFrame(history_3.history).to_csv("/kaggle/working/History-InceptionV3.csv")

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/128
1243/1243 [==============================] - 68s 50ms/step - loss: 1988.8644 - accuracy: 0.4114 - auc: 0.6977 - recall: 0.4114 - precision: 0.4114 - f1_score: 0.4115 - val_loss: 1050.6381 - val_accuracy: 0.4656 - val_auc: 0.7255 - val_recall: 0.4656 - val_precision: 0.4656 - val_f1_score: 0.4656 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 58s 46ms/step - loss: 749.1241 - accuracy: 0.5693 - auc: 0.7791 - recall: 0.5693 - precision: 0.5693 - f1_score: 0.5693 - val_loss: 964.5606 - val_accuracy: 0.5435 - val_auc: 0.7659 - val_recall: 0.5435 - val_precision: 0.5435 - val_f1_score: 0.5435 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 64s 51ms/step - loss: 558.0935 - accuracy: 0.6249 - auc: 0.8078 - recall: 0.6249 - precision: 0.6249 - f1_score: 0.6249 - val_loss: 503.9482 - val_accuracy: 0.5580 - val_auc: 0.7734 - val_recall: 0.5580 - val_precision: 0.5580 - val_f1_

In [19]:
model_4 = create_model_4()

# Fit the model
history_4 = model_4.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_4.save("/kaggle/working/ResNet50.h5")
pd.DataFrame(history_4.history).to_csv("/kaggle/working/History-ResNet50.csv")

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/128
1243/1243 [==============================] - 116s 89ms/step - loss: 0.6640 - accuracy: 0.9287 - auc: 0.9871 - recall: 0.9189 - precision: 0.9452 - f1_score: 0.9304 - val_loss: 0.0868 - val_accuracy: 0.9756 - val_auc: 0.9990 - val_recall: 0.9695 - val_precision: 0.9830 - val_f1_score: 0.9762 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 108s 86ms/step - loss: 0.0521 - accuracy: 0.9866 - auc: 0.9996 - recall: 0.9823 - precision: 0.9911 - f1_score: 0.9866 - val_loss: 0.0512 - val_accuracy: 0.9863 - val_auc: 0.9995 - val_recall: 0.9830 - val_precision: 0.9915 - val_f1_score: 0.9872 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 108s 86ms/step - loss: 0.0320 - accuracy: 0.9925 - auc: 0.9999 - recall: 0.9895 - precision: 0.9952 - f1_score: 0.9923 - val_loss: 0.0476 - val_accuracy: 0.9866 - val_auc: 0.9996 - val_recall: 0.9840 - val_precision: 0.9907 - val_f1_score: 0.98

In [16]:
model_5 = create_model_5()

# Fit the model
history_5 = model_5.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_5.save("/kaggle/working/Xception.h5")
pd.DataFrame(history_5.history).to_csv("/kaggle/working/History-Xception.csv")

83683744/83683744 [==============================] - 1s 0us/step
Epoch 1/128
1243/1243 [==============================] - 343s 270ms/step - loss: 1.1514 - accuracy: 0.6925 - auc: 0.9755 - recall: 0.5067 - precision: 0.9221 - f1_score: 0.6293 - val_loss: 0.5522 - val_accuracy: 0.8583 - val_auc: 0.9947 - val_recall: 0.7515 - val_precision: 0.9616 - val_f1_score: 0.8432 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 123s 98ms/step - loss: 0.5213 - accuracy: 0.8541 - auc: 0.9943 - recall: 0.7748 - precision: 0.9460 - f1_score: 0.8511 - val_loss: 0.3633 - val_accuracy: 0.9013 - val_auc: 0.9969 - val_recall: 0.8469 - val_precision: 0.9656 - val_f1_score: 0.9022 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 122s 98ms/step - loss: 0.3784 - accuracy: 0.8923 - auc: 0.9966 - recall: 0.8406 - precision: 0.9556 - f1_score: 0.8941 - val_loss: 0.2931 - val_accuracy: 0.9181 - val_auc: 0.9975 - val_recall: 0.8792 - val_precision: 0.9673 - val_f1_score: 0.9

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
model_6 = create_model_6()

# Fit the model
history_6 = model_6.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_6.save("/kaggle/working/MobileNetV2.h5")
pd.DataFrame(history_6.history).to_csv("/kaggle/working/History-MobileNetV2.csv")

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/128
1243/1243 [==============================] - 59s 44ms/step - loss: 7.9677 - accuracy: 0.5527 - auc: 0.8396 - recall: 0.5453 - precision: 0.5692 - f1_score: 0.5566 - val_loss: 3.3981 - val_accuracy: 0.6895 - val_auc: 0.9050 - val_recall: 0.6828 - val_precision: 0.7089 - val_f1_score: 0.6956 - lr: 0.0100
Epoch 2/128
1243/1243 [==============================] - 52s 41ms/step - loss: 1.8528 - accuracy: 0.7495 - auc: 0.9377 - recall: 0.7417 - precision: 0.7668 - f1_score: 0.7540 - val_loss: 1.2714 - val_accuracy: 0.7941 - val_auc: 0.9547 - val_recall: 0.7856 - val_precision: 0.8105 - val_f1_score: 0.7979 - lr: 0.0100
Epoch 3/128
1243/1243 [==============================] - 59s 47ms/step - loss: 1.2870 - accuracy: 0.7995 - auc: 0.9567 - recall: 0.7918 - precision: 0.8151 - f1_score: 0.8032 - val_loss: 1.7919 - val_accuracy: 0.7576 - val_auc: 0.9379 - val_recall: 0.7491 - val_precision: 0.7765 - val_f1_score: 0.7624 - 

# **Results**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.saving import load_model

class_names = ['Ahead only', 'Beware of icesnow', 'Bicycles crossing', 'Bumpy road', 'Children crossing', 'Dangerous curve left', 'Dangerous curve right', 'Double curve', 'End no passing veh _ 3.5 tons', 'End of no passing', 'End of speed limit (80kmh)', 'End speed + passing limits', 'General caution', 'Go straight or left', 'Go straight or right', 'Keep left', 'Keep right', 'No entry', 'No passing', 'No passing veh over 3.5 tons', 'No vehicles', 'Pedestrians', 'Priority road', 'Right-of-way at intersection', 'Road narrows on the right', 'Road work', 'Roundabout mandatory', 'Slippery road', 'Speed limit (100kmh)', 'Speed limit (120kmh)', 'Speed limit (20kmh)', 'Speed limit (30kmh)', 'Speed limit (50kmh)', 'Speed limit (60kmh)', 'Speed limit (70kmh)', 'Speed limit (80kmh)', 'Stop', 'Traffic signals', 'Turn left ahead', 'Turn right ahead', 'Veh _ 3.5 tons prohibited', 'Wild animals crossing', 'Yield']

# **EfficientNetV2B0**

In [ ]:
metrics = list(history_1.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_1.history[metric])
        plt.plot(history_1.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **EfficientNetB0**

In [ ]:
metrics = list(history_2.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_2.history[metric])
        plt.plot(history_2.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **InceptionV3**

In [ ]:
metrics = list(history_3.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_3.history[metric])
        plt.plot(history_3.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **ResNet50**

In [ ]:
metrics = list(history_4.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_4.history[metric])
        plt.plot(history_4.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **Xception**

In [ ]:
# metrics = list(history_5.history.keys())

# for metric in metrics:
#     if metric.startswith('val')==False and metric!='lr':
#         plt.figure(figsize=(6, 4))
#         plt.plot(history_5.history[metric])
#         plt.plot(history_5.history['val_'+metric])
#         plt.title('Model '+metric)
#         plt.ylabel(metric)
#         plt.xlabel('Epoch')
#         if metric.endswith('loss'):
#             plt.legend(['Train', 'Validation'], loc='upper right')
#         else:
#             plt.legend(['Train', 'Validation'], loc='lower right')

# plt.show()

# **MobileNetV2**

In [ ]:
metrics = list(history_6.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_6.history[metric])
        plt.plot(history_6.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()